# Introduction
Ce projet a été mené par Valentin Smague et Clément Gadeau dans le cadre d'un cours de deuxième année de l'ENSAE Paris : Python pour la DataScience.
Nous avons choisi pour projet de programmer un modèle qui saurait reconnaître le genre d'un morceau musical. La tâche étant complexe, nous nous sommes retreints aux données rendues disponibles par l'API Spotify, qui nous permettent de nous affranchir de la difficile tâche d'analyse de l'audio.

Peut-on à partir de données quantitatives sur un morceau, prédire le genre musical auquel il appartient ?

Voilà tout l'enjeu de notre projet dont voici une ébauche :

#### __I. Récupérer et traiter les données__
  - Requêter l'API Spotify
  - Re-traiter la variable de genre
  - Constituer une base de donnée qui retranscrit bien l'étendu de l'univers musical

#### __II. Visualiser__
  - Vérifier la bonne répartition des poids dans notre base
  - Comprendre les relations entre les variables
  - Comprendre les relations entre les variables et le genre

#### __III. Prédire le genre__
  - Random Forest
  - Valentiiiiin c:

---

# __I. Récupérer et traiter les données__

Nous aurons avant tout besoin d'importer quelques modules et packages

In [71]:
import requests
import pandas as pd
from base64 import b64encode
import ast


### __Requêter l'API__

Pour constituer notre base de données, nous utilisons l'API Spotify. Le premier problème auquel nous sommes confrontés est que Spotify ne met pas à disposition une quelconque base de donnée déjà constituée ; pas même une liste de morceaux. 
Cependant nous pouvons requêter l'API pour obtenir des informations sur : un morceau donné, un artiste donné, ou une playlist donnée. C'est ce que nous avons choisi de faire. 

Nous sélectionnons donc une playlist de 10 000 morceaux déjà faite par un utilisateur Spotify, afin d'obtenir des informations sur chaque morceau qui la compose : titre, id, artist, artist id. 
Ensuite, nous récupérons les informations qui nous intéressent sur chaque morceaux : les track features, qui sont un tas d'indices quantitatifs sur chaque morceau.
Enfin, nous récupérons le genre qui n'est associé qu'à l'artiste, et pas au morceau.

D'abord, nous devons obtenir le token d'accès à l'API. Les logins sont dans logins.txt


In [173]:
def get_access_token():
    """
    Returns the access token for Spotify API
    """
    logins = "Data/data/logins.txt"
    with open(logins, "r") as file:
        client_id = str(file.readline().strip())
        client_secret = str(file.readline())
    file.close()
    
    token_url = "https://accounts.spotify.com/api/token"

    # Concatenates client-id and client_secret, then encodes them in base64
    credentials = b64encode(f"{client_id}:{client_secret}".encode()).decode('utf-8')

    # Requests headers
    headers = {'Authorization': f'Basic {credentials}','Content-Type': 'application/x-www-form-urlencoded',}

    # Requests body
    data = {'grant_type': 'client_credentials',}

    # Does the Request
    response = requests.post(token_url, headers=headers, data=data)

    # Makes sure the response does not mean error.
    if response.status_code == 200:
        # We obtain the access token
        access_token = response.json().get('access_token')
        return(access_token)
    else:
        print(f"Error during the token request : {response.status_code} - {response.text}")
        return(None)


access_token = get_access_token()

### __Constituer un premier dataset__

Ensuite nous utilisons ce token pour notre première requête : obtenir des informations sur une playlist de 10 000 morceaux. Nous avons choisi ce genre de playlist faite pour pouvoir être utilisée dans l'API, et nous espérons donc qu'elle fasse état d'une bonne répartition des genres.

Nous entrons l'id de la playlist en question.
Nous récupérons les informations de la playlist relatives aux morceaux dans get_all_playlist_tracks.
Avec les informations obtenues précédemment, nous créons avec get_track_id_and_artist un dataframe qui pour chaque morceau de la playlist en donne le titre, l'id, l'artist, l'artist id.

In [58]:
playlist_id = '04ZwFco4KsjgPlVMtzwfgS'

def get_all_playlist_tracks(playlist_id, access_token):
    playlist_url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    headers = {'Authorization': 'Bearer ' + access_token}
    params = {'offset': 0, 'limit': 100}  # Limits the response to 100 each time.

    all_tracks = []

    while True:
        response = requests.get(playlist_url, headers=headers, params=params)

        if response.status_code == 200:
            playlist_data = response.json()
            tracks = playlist_data['items']
            all_tracks.extend(tracks)

            # Check if there are other tracks to recover
            if playlist_data['next']:
                # Update the offset to get the following page.
                params['offset'] += params['limit']
            else:
                break
        else:
            print(f"Error retrieving tracks from playlist. Error code : {response.status_code}")
            return None

    return all_tracks


def get_track_id_and_artist(tracks):
    if tracks:
        title = []
        track_id = []
        artist = []
        artist_id = []
        for track in tracks:
            title.append(track['track']['name'])
            track_id.append(track['track']['id'])
            artist.append(track['track']['artists'][0]['name'])
            artist_id.append(track['track']['artists'][0]['id'])
        df = pd.DataFrame({'Title':title})
        df['track_id'] = track_id
        df['artist'] = artist
        df['artist_id'] = artist_id
        return df
    else:
        print("No track found.")
        return None

all_tracks = get_all_playlist_tracks(playlist_id, access_token)
df_init = get_track_id_and_artist(all_tracks)

Voilà ce à quoi ressemble ce premier dataframe :

In [59]:
df_init.head()

,Title,track_id,artist,artist_id
0,Act Naturally,3JWEMzwpcWCvu4Qw1BIbYi,Buck Owens,2FMZn5P3WATd7Il6FgPJNu
1,How Ya Do Dat,1KLhUURHRl72xGO5A94lme,Young Bleed,5GQgxUq4MOuXXV99WrRuev
2,Bury Me Alive,5SbJacDwoaouWkGxG1TrHq,We Are The Fallen,4V0jSDdGGQ2VhYuSOjBUjI
3,Army of Me,3vEmpJQC2p8h0U3OiBufQm,Björk,7w29UYBi0qsHi5RTcv3lmA
4,Dynamite,1DqdF42leyFIzqNDv9CjId,Taio Cruz,6MF9fzBmfXghAz953czmBC


Puis nous voulons pour chacun de ces morceaux des informations quantitatives. Pour cela nous utilisons get_track_features. On fait une seule et même requête pour 100 morceaux à la fois (c'est le maximum imposé par l'API).

In [60]:
def get_track_features(track_id, headers):
    """
    Returns a dictionnary for the track features.
    track_id my be a list of under 100 tracks or just a string for one single track.
    """
    
    str_track_id = ','.join(track_id)

    params = {'ids': str_track_id}
    features_url = "https://api.spotify.com/v1/audio-features"
    response = requests.get(features_url, headers=headers,params=params)
    print(response)

    # Verify the response: 
    if response.status_code == 200:
        # La réponse est au format JSON, imprimez toutes les caractéristiques
        data = response.json()
        return(data['audio_features'])
    else:
        print(f"Erreur lors de la requête : {response.status_code} - {response.text}")
        return([None]*len(track_id))

On utilise la fonction pour obtenir les caractéristiques de tous les morceaux de notre dataset initial en opérant 100 par 100.

In [61]:
track_list = df_init['track_id'].tolist()
track_number = len(track_list)
print("There are ", track_number, " tracks")

nb_use_token_track = track_number // 100 + int(track_number % 100 != 0)
track_features_list = []

headers = {'Authorization': f'Bearer {access_token}',}
for k in range(nb_use_token_track-1):
    track_features_list = track_features_list + get_track_features(track_list[100*k:100*(k+1)],headers)
track_features_list = track_features_list + get_track_features(track_list[(nb_use_token_track-1)*100:],headers)

df_init['track_features'] = track_features_list

df_init.head()

There are  9999  tracks
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Respon

,Title,track_id,artist,artist_id,track_features
0,Act Naturally,3JWEMzwpcWCvu4Qw1BIbYi,Buck Owens,2FMZn5P3WATd7Il6FgPJNu,"{'danceability': 0.552, 'energy': 0.474, 'key'..."
1,How Ya Do Dat,1KLhUURHRl72xGO5A94lme,Young Bleed,5GQgxUq4MOuXXV99WrRuev,"{'danceability': 0.788, 'energy': 0.451, 'key'..."
2,Bury Me Alive,5SbJacDwoaouWkGxG1TrHq,We Are The Fallen,4V0jSDdGGQ2VhYuSOjBUjI,"{'danceability': 0.529, 'energy': 0.882, 'key'..."
3,Army of Me,3vEmpJQC2p8h0U3OiBufQm,Björk,7w29UYBi0qsHi5RTcv3lmA,"{'danceability': 0.462, 'energy': 0.677, 'key'..."
4,Dynamite,1DqdF42leyFIzqNDv9CjId,Taio Cruz,6MF9fzBmfXghAz953czmBC,"{'danceability': 0.754, 'energy': 0.804, 'key'..."


Seulement, maintenant on a une colonne features qui constient elle même plusieurs colonnes. C'est pourquoi on l'extrait.

In [62]:
from Script.functions import get_features_labels
Features = get_features_labels(headers)
# We get a list of the features labels.

for feature in Features:
    df_init[feature] = df_init['track_features'].apply(lambda x: x.get(feature))
df_init.drop(columns=['track_features'], inplace = True)

df_init.head()

,Title,track_id,artist,artist_id,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Act Naturally,3JWEMzwpcWCvu4Qw1BIbYi,Buck Owens,2FMZn5P3WATd7Il6FgPJNu,0.552,0.474,7,-6.847,1,0.0275,...,0.1730,0.700,92.156,audio_features,3JWEMzwpcWCvu4Qw1BIbYi,spotify:track:3JWEMzwpcWCvu4Qw1BIbYi,https://api.spotify.com/v1/tracks/3JWEMzwpcWCv...,https://api.spotify.com/v1/audio-analysis/3JWE...,143693,4
1,How Ya Do Dat,1KLhUURHRl72xGO5A94lme,Young Bleed,5GQgxUq4MOuXXV99WrRuev,0.788,0.451,1,-9.313,1,0.3180,...,0.1340,0.302,89.970,audio_features,1KLhUURHRl72xGO5A94lme,spotify:track:1KLhUURHRl72xGO5A94lme,https://api.spotify.com/v1/tracks/1KLhUURHRl72...,https://api.spotify.com/v1/audio-analysis/1KLh...,271067,4
2,Bury Me Alive,5SbJacDwoaouWkGxG1TrHq,We Are The Fallen,4V0jSDdGGQ2VhYuSOjBUjI,0.529,0.882,11,-2.962,0,0.0444,...,0.3590,0.179,119.974,audio_features,5SbJacDwoaouWkGxG1TrHq,spotify:track:5SbJacDwoaouWkGxG1TrHq,https://api.spotify.com/v1/tracks/5SbJacDwoaou...,https://api.spotify.com/v1/audio-analysis/5SbJ...,286640,4
3,Army of Me,3vEmpJQC2p8h0U3OiBufQm,Björk,7w29UYBi0qsHi5RTcv3lmA,0.462,0.677,6,-9.262,1,0.0327,...,0.2260,0.468,172.192,audio_features,3vEmpJQC2p8h0U3OiBufQm,spotify:track:3vEmpJQC2p8h0U3OiBufQm,https://api.spotify.com/v1/tracks/3vEmpJQC2p8h...,https://api.spotify.com/v1/audio-analysis/3vEm...,234333,4
4,Dynamite,1DqdF42leyFIzqNDv9CjId,Taio Cruz,6MF9fzBmfXghAz953czmBC,0.754,0.804,4,-3.177,1,0.0853,...,0.0329,0.818,119.968,audio_features,1DqdF42leyFIzqNDv9CjId,spotify:track:1DqdF42leyFIzqNDv9CjId,https://api.spotify.com/v1/tracks/1DqdF42leyFI...,https://api.spotify.com/v1/audio-analysis/1Dqd...,203867,4


Enfin, nous voulons accéder au genre de chaque morceau. L'API associe le genre d'un morceau à son artiste. En outre, l'API n'associe pas un unique genre à un morceau, mais une liste de genres. De la même façon que pour obtenir les features, on émet une requête pour obtenir le genre de l'artiste.

In [63]:
def get_artists_genres(artist_ids, headers):
    # Convert the list of artist IDs to a comma-separated string
    artists_str = ",".join(artist_ids)

    # Endpoint to obtain information on several artists
    endpoint = f"https://api.spotify.com/v1/artists?ids={artists_str}"

    # Make a GET request to the Spotify API
    response = requests.get(endpoint, headers=headers)

    # Check if request was successful (status 200 OK)
    if response.status_code == 200:
        # Analyze the JSON response
        data = response.json()

        # Recover each artist's genre
        all_genres = []
        for artist_data in data["artists"]:
            genres = artist_data.get("genres", [])
            all_genres.append(genres)

        return all_genres
    else:
        # Display error message if query failed
        print(f"Error {response.status_code}: Unable to obtain artist genres.")
        return [None]*len(artist_ids)

In [64]:
artist_list = df_init['artist_id'].tolist()
nb_artist = len(artist_list)
print("There are ", nb_artist, " artists")

nb_use_token_artist = nb_artist // 50 + int(nb_artist % 50 != 0)
artist_genres_list = []

for k in range(nb_use_token_artist-1):
    artist_genres_list = artist_genres_list + get_artists_genres(artist_list[50*k:50*(k+1)], headers)
    print(k/nb_use_token_artist)
artist_genres_list = artist_genres_list + get_artists_genres(artist_list[(nb_use_token_artist-1)*50:], headers)
df_init['genres'] = artist_genres_list

df_init.head()

There are  9999  artists
0.0
0.005
0.01
0.015
0.02
0.025
0.03
0.035
0.04
0.045
0.05
0.055
0.06
0.065
0.07
0.075
0.08
0.085
0.09
0.095
0.1
0.105
0.11
0.115
0.12
0.125
0.13
0.135
0.14
0.145
0.15
0.155
0.16
0.165
0.17
0.175
0.18
0.185
0.19
0.195
0.2
0.205
0.21
0.215
0.22
0.225
0.23
0.235
0.24
0.245
0.25
0.255
0.26
0.265
0.27
0.275
0.28
0.285
0.29
0.295
0.3
0.305
0.31
0.315
0.32
0.325
0.33
0.335
0.34
0.345
0.35
0.355
0.36
0.365
0.37
0.375
0.38
0.385
0.39
0.395
0.4
0.405
0.41
0.415
0.42
0.425
0.43
0.435
0.44
0.445
0.45
0.455
0.46
0.465
0.47
0.475
0.48
0.485
0.49
0.495
0.5
0.505
0.51
0.515
0.52
0.525
0.53
0.535
0.54
0.545
0.55
0.555
0.56
0.565
0.57
0.575
0.58
0.585
0.59
0.595
0.6
0.605
0.61
0.615
0.62
0.625
0.63
0.635
0.64
0.645
0.65
0.655
0.66
0.665
0.67
0.675
0.68
0.685
0.69
0.695
0.7
0.705
0.71
0.715
0.72
0.725
0.73
0.735
0.74
0.745
0.75
0.755
0.76
0.765
0.77
0.775
0.78
0.785
0.79
0.795
0.8
0.805
0.81
0.815
0.82
0.825
0.83
0.835
0.84
0.845
0.85
0.855
0.86
0.865
0.87
0.875
0.88
0.885
0.89


,Title,track_id,artist,artist_id,danceability,energy,key,loudness,mode,speechiness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genres
0,Act Naturally,3JWEMzwpcWCvu4Qw1BIbYi,Buck Owens,2FMZn5P3WATd7Il6FgPJNu,0.552,0.474,7,-6.847,1,0.0275,...,0.700,92.156,audio_features,3JWEMzwpcWCvu4Qw1BIbYi,spotify:track:3JWEMzwpcWCvu4Qw1BIbYi,https://api.spotify.com/v1/tracks/3JWEMzwpcWCv...,https://api.spotify.com/v1/audio-analysis/3JWE...,143693,4,"[bakersfield sound, classic country pop, count..."
1,How Ya Do Dat,1KLhUURHRl72xGO5A94lme,Young Bleed,5GQgxUq4MOuXXV99WrRuev,0.788,0.451,1,-9.313,1,0.3180,...,0.302,89.970,audio_features,1KLhUURHRl72xGO5A94lme,spotify:track:1KLhUURHRl72xGO5A94lme,https://api.spotify.com/v1/tracks/1KLhUURHRl72...,https://api.spotify.com/v1/audio-analysis/1KLh...,271067,4,"[baton rouge rap, dirty south rap]"
2,Bury Me Alive,5SbJacDwoaouWkGxG1TrHq,We Are The Fallen,4V0jSDdGGQ2VhYuSOjBUjI,0.529,0.882,11,-2.962,0,0.0444,...,0.179,119.974,audio_features,5SbJacDwoaouWkGxG1TrHq,spotify:track:5SbJacDwoaouWkGxG1TrHq,https://api.spotify.com/v1/tracks/5SbJacDwoaou...,https://api.spotify.com/v1/audio-analysis/5SbJ...,286640,4,[pixie]
3,Army of Me,3vEmpJQC2p8h0U3OiBufQm,Björk,7w29UYBi0qsHi5RTcv3lmA,0.462,0.677,6,-9.262,1,0.0327,...,0.468,172.192,audio_features,3vEmpJQC2p8h0U3OiBufQm,spotify:track:3vEmpJQC2p8h0U3OiBufQm,https://api.spotify.com/v1/tracks/3vEmpJQC2p8h...,https://api.spotify.com/v1/audio-analysis/3vEm...,234333,4,"[art pop, electronica, experimental pop, exper..."
4,Dynamite,1DqdF42leyFIzqNDv9CjId,Taio Cruz,6MF9fzBmfXghAz953czmBC,0.754,0.804,4,-3.177,1,0.0853,...,0.818,119.968,audio_features,1DqdF42leyFIzqNDv9CjId,spotify:track:1DqdF42leyFIzqNDv9CjId,https://api.spotify.com/v1/tracks/1DqdF42leyFI...,https://api.spotify.com/v1/audio-analysis/1Dqd...,203867,4,"[dance pop, pop, pop rap]"


In [65]:
#path = '/home/onyxia/work/Python-pour-la-Data-Science/Data/data/Titles2.csv'
#df_init.to_csv(path, index=False)

On nettoie alors notre base de donnée pour enlever tous les éléments sans genre. 

In [129]:
df_cleaned = df_init.copy()
print("Tracks number before cleaning ", df_cleaned.shape[0])
df_cleaned = df_cleaned.dropna(subset=['genres'])
df_cleaned = df_cleaned[df_cleaned['genres'].apply(lambda x: isinstance(x, list) and any(isinstance(item, str) and any(c.isalpha() for c in item) for item in x))]
#df_cleaned = df_cleaned[df_cleaned['genres'].apply(lambda x: (isinstance(x, str) and any(c.isalpha() for c in x)))]
#If the data_frame is red from a csv file.
print("Tracks number after cleaning ", df_cleaned.shape[0])
print(f"Thus there were {9999 - df_cleaned.shape[0]} tracks without genre")

df_cleaned = df_cleaned.drop(columns=['Title', 'track_id', 'artist', 'artist_id', 'type', 'id', 'uri', 'track_href', 'analysis_url'])
df_cleaned.head()


Tracks number before cleaning  9999
Tracks number after cleaning  9350
Thus there were 649 tracks without genre


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genres
0,0.552,0.474,7,-6.847,1,0.0275,0.33600,0.000000,0.1730,0.700,92.156,143693,4,"[bakersfield sound, classic country pop, count..."
1,0.788,0.451,1,-9.313,1,0.3180,0.11800,0.000000,0.1340,0.302,89.970,271067,4,"[baton rouge rap, dirty south rap]"
2,0.529,0.882,11,-2.962,0,0.0444,0.02080,0.000013,0.3590,0.179,119.974,286640,4,[pixie]
3,0.462,0.677,6,-9.262,1,0.0327,0.00786,0.000072,0.2260,0.468,172.192,234333,4,"[art pop, electronica, experimental pop, exper..."
4,0.754,0.804,4,-3.177,1,0.0853,0.00332,0.000000,0.0329,0.818,119.968,203867,4,"[dance pop, pop, pop rap]"


### __Nettoyer le dataset : identifier un genre unique pour chaque morceau__

Le problème est maintenant que l'on a une liste de genre pour chaque morceau, et pas un unique genre par morceau. Plutôt que de tronquer la liste à son premier élément, nous préférons pour chaque liste vérifier s'il n'y a pas de sous-genres dans la liste. 

Un sous genre sera défini comme un genre qui contient un autre genre dans son nom. Par exemple, nous considérerons le hardrock comme un sous genre du rock. Par contre le pop-rock sera considéré simultanément comme un sous genre du rock et de la pop. On veut donc un dictionnaire qui à chaque genre associe une liste de ses genres parents.

On crée 3 fonctions
- get_genre_list qui prend en argument un dataset, et qui renvoie la liste des genres musicaux dans le dataset.
- reduce_genres_list qui prend en argument une liste des genres musicaux, et qui renvoie un dictionnaire qui associe à sous-genre ses genres parents.
- remake_genre_list qui prend en argument une liste de genres, et qui renvoie le grand genre le plus fréquent dans les grands genres associés au éléments de la liste de genres.

In [130]:
def get_genres_list(data_set):
    """Returns the list of genres that appear in data_set"""
    union_genres = set()
    for genre_str in data_set['genres']:
        #if pd.notna(genre_str):
            # We use ast.literal_eval to get a list from a string of a list
        genre_list = genre_str #ast.literal_eval(genre_str)
        union_genres = union_genres.union(set(genre_list))
    return(list(union_genres))


def reduce_genres_list(list_of_genres):
    """Returns a dictionnary of genres parenthood for a list of genre"""
    sorted_list = sorted(list_of_genres, key=lambda x: len(x))
    visited_words = []
    dict = {}
    for genre in sorted_list:
        dict[genre] = [genre]
        for word in visited_words:
            if word in genre and word != genre:
                dict[genre] += [word]
        if len(dict[genre]) > 1:
            dict[genre].remove(genre)
        visited_words += [genre]
    return dict


def remake_genre_list(genre_list, dict):
    """
    Returns the most frequent genre from a list of genre.
    It will be used for each tracks' list of genres.
    """
    new_list = []
    for genre in genre_list: #ast.literal_eval(genre_list):
        new_list += dict[genre]
    most_frequent_genre = max(set(new_list), key=new_list.count)
    return most_frequent_genre

On crée une liste des genres présents. Puis on l'utilise pour obtenir le dictionnaire des parentées de genres. Ensuite on transforme la colonne des genres pour que chaque élément soit le grand genre le plus parlant pour chaque morceau.

On affiche ensuite un data frame qui montre pour chaque grand genre final le nombre d'occurrence qu'il a dans la base de donnée ainsi modifiée. On observe que certains genres sont très représentés mais qu'assez vite on arrive sur des genres peu fréquents.

In [175]:
df_cleaned_copy = df_cleaned.copy()
dict = reduce_genres_list(get_genres_list(df_cleaned_copy))
df_cleaned_copy['genres'] = df_cleaned_copy.apply(lambda row: remake_genre_list(row['genres'], dict), axis=1)
genres_occurrence = df_cleaned_copy['genres'].value_counts().reset_index()
genres_occurrence.head()

{'technical black metal', 'arkansas country', 'german blues', 'deep new americana', 'new orleans indie', 'tropical alternativo', 'folk-pop', 'gen z singer-songwriter', 'industrial rock', 'fantasy', 'deep melodic hard rock', 'afro-cuban percussion', 'acid techno', 'cascadian black metal', 'indie valenciana', 'finnish dance pop', 'vaporwave', 'kelowna bc indie', 'thai folk', 'fremantle indie', 'canadian old school hip hop', 'apostolic worship', 'soft rock', 'umea hardcore', 'texas pop punk', 'calgary indie', 'latin metal', 'relaxative', 'post-grunge', 'orebro indie', 'avant-garde jazz', 'ska revival', 'indian edm', 'jazz trio', 'madchester', 'escape room', 'ruta destroy', 'covertronica', 'idol', 'chapman stick', 'latinx alternative', 'smooth soul', 'darkstep', 'florida drill', 'piano rock', 'riddim', 'la indie', 'asheville indie', 'retro soul', 'electro swing', 'future ambient', 'chicano rap', 'neurostep', 'trap queen', 'miami indie', 'raw techno', 'german thrash metal', 'easycore', 'bel

,genres,count
0,pop,1953
1,rock,980
2,hip hop,563
3,metal,515
4,country,384


In [165]:
saved_genres = genres_occurrence.iloc[:12, 0].tolist()
df_filtered = df_cleaned_copy[df_cleaned_copy['genres'].isin(saved_genres)]
print(f"There are {df_filtered.shape[0]} tracks associated to one of the top 12 most listened genres")


There are 6016 tracks associated to one of the top 12 most listened genres


### __Interroger nos résultats dans une démarche scientifique__

On remarque que les genres ne sont pas du tout représentés en quantités équivalentes et que certaines le sont beaucoup plus que d'autres. Alors notre playlist ne nous fournira pas un dataset assez bon.

__Néanmoins, ces péripéties nous ont permis d'avoir une liste de larges genres musicaux pour constituer notre base.__ 

En effet : avant, nous aurions certainement eu un biais dans la réalisation de notre "liste de genres à avoir". Mais ces dernières recherchent nous ont permis de déterminer des larges groupes de genres musicaux qui sont représentatifs de l'ensemble des genres écoutés sur la plateforme spotify. Nous allons à présent créer une base de données plus artisanale en concaténant des datasets de morceaux d'un certain genre. 

### __Constituer un dataset muris de nos réflexions__

A présent nous allons, en prenant les plus larges genres les plus courants, constituer une base de donnée équilibrée en terme d'occurrence des différents genres.

Pour cela nous allons utiliser des playlists spotify de l'ordre de 1000 sons qui sont déjà associées à un genre en particulier.

La méthode est la même que précédemment, à ceci près que nous associons nous même le genre à chaque morceau, en fonction de la playlist de provenance du morceau.

In [174]:
playlist_id_pop = '04ZwFco4KsjgPlVMtzwfgS'
playlist_id_rock = '04ZwFco4KsjgPlVMtzwfgS'
playlist_id_hip_hop = '04ZwFco4KsjgPlVMtzwfgS'
playlist_id_metal = '04ZwFco4KsjgPlVMtzwfgS'
playlist_id_country = '04ZwFco4KsjgPlVMtzwfgS'
playlist_id_rap = '04ZwFco4KsjgPlVMtzwfgS'
playlist_id_soul = '04ZwFco4KsjgPlVMtzwfgS'
playlist_id_jazz = '04ZwFco4KsjgPlVMtzwfgS'
playlist_id_house = '04ZwFco4KsjgPlVMtzwfgS'
playlist_id_punk = '04ZwFco4KsjgPlVMtzwfgS'
playlist_id_r_and_b = '04ZwFco4KsjgPlVMtzwfgS'
playlist_id_blues = '04ZwFco4KsjgPlVMtzwfgS'
playlist_id_new_wave = '04ZwFco4KsjgPlVMtzwfgS'
playlist_id_folk = '04ZwFco4KsjgPlVMtzwfgS'
playlist_id_electro = '04ZwFco4KsjgPlVMtzwfgS'
playlist_id_funk = '04ZwFco4KsjgPlVMtzwfgS'

playlist_id_list = [playlist_id_pop, playlist_id_rock, playlist_id_rock, playlist_id_hip_hop, playlist_id_metal, playlist_id_country, playlist_id_rap, playlist_id_soul, playlist_id_jazz, playlist_id_house, playlist_id_punk, playlist_id_r_and_b, playlist_id_blues, playlist_id_new_wave, playlist_id_folk, playlist_id_electro, playlist_id_funk]
playlist_df_list = []

for id in playlist_id_list:
    all_tracks = get_all_playlist_tracks(id, access_token)
    df = get_track_id_and_artist(all_tracks)
    track_list = df['track_id'].tolist()
    track_number = len(track_list)

    nb_use_token_track = track_number // 100 + int(track_number % 100 != 0)
    track_features_list = []

    headers = {'Authorization': f'Bearer {access_token}',}
    for k in range(nb_use_token_track-1):
        track_features_list = track_features_list + get_track_features(track_list[100*k:100*(k+1)],headers)
    track_features_list = track_features_list + get_track_features(track_list[(nb_use_token_track-1)*100:],headers)

    df['track_features'] = track_features_list
    for feature in Features:
        df[feature] = df['track_features'].apply(lambda x: x.get(feature))
    df.drop(columns=['track_features'], inplace = True)

    artist_list = df['artist_id'].tolist()
    nb_artist = len(artist_list)

    nb_use_token_artist = nb_artist // 50 + int(nb_artist % 50 != 0)
    artist_genres_list = []

    for k in range(nb_use_token_artist-1):
        artist_genres_list = artist_genres_list + get_artists_genres(artist_list[50*k:50*(k+1)], headers)
        print(k/nb_use_token_artist)
    artist_genres_list = artist_genres_list + get_artists_genres(artist_list[(nb_use_token_artist-1)*50:], headers)
    df['genres'] = artist_genres_list

    df_copy = df.copy()
    df_copy = df_copy.dropna(subset=['genres'])
    df_copy = df_copy[df_copy['genres'].apply(lambda x: isinstance(x, list) and any(isinstance(item, str) and any(c.isalpha() for c in item) for item in x))]


    df_copy = df_copy.drop(columns=['Title', 'track_id', 'artist', 'artist_id', 'type', 'id', 'uri', 'track_href', 'analysis_url'])
    playlist_df_list.append(df_copy.copy())

print(playlist_df_list)

                   Title                track_id             artist  \
0          Act Naturally  3JWEMzwpcWCvu4Qw1BIbYi         Buck Owens   
1          How Ya Do Dat  1KLhUURHRl72xGO5A94lme        Young Bleed   
2          Bury Me Alive  5SbJacDwoaouWkGxG1TrHq  We Are The Fallen   
3             Army of Me  3vEmpJQC2p8h0U3OiBufQm              Björk   
4               Dynamite  1DqdF42leyFIzqNDv9CjId          Taio Cruz   
...                  ...                     ...                ...   
9994        Stick Season  0mflMxspEfB0VbI1kyLiAv         Noah Kahan   
9995  Boy's a liar Pt. 2  6AQbmUe0Qwf5PZnt4HmTXv     PinkPantheress   
9996   Feathered Indians  2tgQaL85WoRfgEa4hFQgrE     Tyler Childers   
9997         Playing God  4S0zagxseJHOL5ZVnTQA6X           Polyphia   
9998   Is It Really You?  4OmlsAT8r4q9vPFBvfYgyZ             Loathe   

                   artist_id  
0     2FMZn5P3WATd7Il6FgPJNu  
1     5GQgxUq4MOuXXV99WrRuev  
2     4V0jSDdGGQ2VhYuSOjBUjI  
3     7w29UYBi0qsHi5RTc

KeyboardInterrupt: 

On peut maintenant concaténer tous ces dataset en une seule et même base de donnée qui sera la base sur laquelle nous travaillerons.

In [ ]:
Dataset_Genres = pd.concat(playlist_df_list, ignore_index=True).drop_duplicates()

# __II. Visualiser__

A présent nous allons tacher de mettre en relief les données de notre base de données, en les visualisant sur différents graphes.

1. __Bonne répartition des poids : Matrice des corrélations__

Dans un premier temps nous vérifions si nos variables sont corrélées. C'est une étape importante et nécessaire si l'on veut que notre modèle travaille sans peine ! 


2. Graphes divers



Camembert de la répartition des genres.

Pour chaque genre ; sortir Espérance, Variance ? dans histogramme

Pour chaque genre : faire une matrice des corrélations du dataset restreint au genre en particulier ?

Pour chaque genre : faire un graphe de telle variable en fonction de telle autre ?

# __III. Prédictions__

 1. __Random Forest__


2. __XGboost__

3. __Résultats__